<div style="text-align: center; color: red; font-size: 20px;">
    BASICS
</div>

```
docker     # list of all commands
docker -v  # version of the docker

```

<div style="text-align: center; color: red; font-size: 20px;">
    IMAGES
</div>

```
# playstore of docker images
hub.docker.com

docker pull <image_name>
docker pull <image_name>:<version>
```

```
docker images | docker image ls
docker rmi -f <image_id>
```

```
#Publishing docker image
docker login
docker push <image_name>
```

<div style="text-align: center; color: red; font-size: 20px;">
    CONTAIER
</div>

```
docker ps -a    |  docker container ls -a : all containers
docker ps       |  docker container ls    : running container
docker container stats                    : performance stats of all containers

docker container start <container_id>
docker container stop <container_id>
docker container rm -f <container_id> <container_id> <container_id>

docker contianer logs -f <container_id>
docker container top <container_id>          : top process running inside the container
docker container inspect <container_id>      : details of docker container config

```

#### Make Changes in docker image

```
docker exec -it <container_id> <command>   # Allows us to run addictional command over already running container
                               /bin/bash   # Use defualt bash if it didnt work go for sh
                               /bin/sh
Ctrl + d : exit from the container

docker commit <container_id> <image_id>
```

#### Volume Mounting

```
Persitant data store in local even after docker container stops
Example : logs of the code


docker volumn ls
docker volumne create <vol_name>  # Defuault_directory : /var/lib/docker/volumnes
docker volumne rm <vol_name>
docker volumne prune

```

#### container network

```
for the interaction of two containers : docker network is need to be same

docker network ls
docker network create <network_name>
docker network rm <network_name>
docker network inspect bridge/host 

drivers in network
bridge: defualt (port mapping is required)
host : container has it no ip address it uses host machine ip address  (port mapping is not required)
null : to create compleatly isolated network (container has no internet access)
```

<div style="text-align: center; color: red; font-size: 20px;">
    OTHER
</div>

```
docker system prune : Remove junk in Docker
```

<div style="text-align: center; color: red; font-size: 20px;">
    PROJECT
</div>

### requirement.txt 

```
pip freeze > requirements.txt
```

### .dockerignore

```
# Ignore Python cache and compiled files
__pycache__/
*.py[cod]
*.pyo

# Ignore Jupyter Notebook checkpoints
.ipynb_checkpoints

# Ignore environment and virtualenv folders
.env
.venv
env/
venv/

# Ignore Docker-related files that shouldn't be copied
.dockerignore
Dockerfile

# Ignore version control directories
.git
.gitignore

# Ignore OS-specific files
.DS_Store
Thumbs.db

# Ignore IDE/editor configs
.vscode/
.idea/

# Ignore logs and temporary files
*.log
*.tmp
*.swp
*.bak

# Ignore test coverage and build artifacts
coverage/
htmlcov/
dist/
build/
*.egg-info/

# Ignore node_modules if you're using Node.js
node_modules/

# Ignore logs
**/*.log
```

### Dockerfile

```
##### Set Base Method #####
FROM ubuntu:20.04

##### Take Input from image_build.sh   ########
ARG user_name
ARG uid
ARG gid
ARG home_dir
ARG work_dir

##### Set environment variables early #####
ENV DEBIAN_FRONTEND=noninteractive
ENV TZ=Asia/Kolkata
ENV PATH=${home_dir}/.local/bin:${home_dir}:$PATH

##### Install required packages (python, tzdata, ping, curl) #####
RUN apt-get update && \
    apt-get install -y python3-pip tzdata iputils-ping curl

##### Configure the timezone non-interactively #####
RUN echo $TZ > /etc/timezone && \
    dpkg-reconfigure -f noninteractive tzdata

##### Create new user & group and switch from root #####
RUN groupadd -g $gid $user_name && \
    useradd -ms /bin/sh $user_name --uid $uid --gid $gid && \
    mkdir -p $home_dir $work_dir && \
    chown -R $user_name:$user_name $home_dir $work_dir

USER $user_name

##### Set working directory dynamically #####
WORKDIR ${work_dir}

##### Install Python requirements #####
COPY requirements.txt requirements.txt
RUN pip install --user -r requirements.txt

##### Copy all project files #####
COPY . .

##### Default command #####
CMD ["/bin/sh"]
```

### image_build.sh --> sh image_build.sh

```
#!/bin/sh

# ---------------------- Configuration ----------------------
USER_NAME="analytics"
UID_VAL=$(id -u "$USER_NAME")
GID_VAL=$(id -g "$USER_NAME")

HOME_DIR="/home/analytics"                          # Directory where your python libraries will be stored
WORKING_DIR="/home/analytics/telecalling_project"   # change this dir to local config base_dir

IMAGE_NAME="telecalling"
IMAGE_VERSION="1.0.0"


# Docker build command
docker build \
  --build-arg uid="$UID_VAL" \
  --build-arg gid="$GID_VAL" \
  --build-arg user_name="$USER_NAME" \
  --build-arg home_dir="$HOME_DIR" \
  --build-arg work_dir="$WORKING_DIR" \
  -t "${IMAGE_NAME}:${IMAGE_VERSION}" \
  .
```

### container_run.sh ---> sh container_run.sh

```
#!/bin/sh

IMAGE_NAME="telecalling"
IMAGE_VERSION="1.0.0"
CONTAINER_NAME="${IMAGE_NAME}_${IMAGE_VERSION}_container"
COMMAND="/usr/bin/python3 /driver_.py -c ROI"

# Volume mount variables
HOST_PATH="/your/host/path"                
CONTAINER_PATH="/your/container/path"      

# Docker run command
timeout 1200 docker run \
  --stop-timeout=1200 \
  --rm \
  -itd \
  
  -v "${HOST_PATH}:${CONTAINER_PATH}" \
  
  -p 8080:8080 \
  --network bridge \
  
  -e \
  
  --name "$CONTAINER_NAME" \
  "${IMAGE_NAME}:${IMAGE_VERSION}" \
  
  sh -c "$COMMAND"

```

### .yaml file for container run

```
# Docker Compose (.yaml file) : tool to run docker container instead of running directly on terminal
# all the container defined on .yaml file , by default they all run all same network , that why there no parameter of network

services:
    <container_name>:
        image:<image_name>
        ports:
        - <host_port>:<container_port>
        environment:
            <ENV_VARIABLE1>:<value1>
            <ENV_VARIABLE2>:<value2>
        volumes:
        -<host_dir>:<image_directory>

docker compose -f <.yaml> up -d
docker compose -f <.yaml> down
```

```
services:
  telecalling:
    container_name: telecalling_1.0.0_container
    image: telecalling:1.0.0
    command: sh -c "/usr/bin/python3 /driver_.py -c ROI"
    volumes:
      - /your/host/path:/your/container/path  # Replace with actual paths
    ports:
      - "8080:8080"
    networks:
      - default
    environment:
      # Add your environment variables below (currently empty in your script)
      # EXAMPLE_VAR: "value"
    restart: "no"
```